In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
!ls

 dic487-587.zip     mapping.csv  'Part1 (1).ipynb'   sample.csv   train.csv
 genre_lables.csv   p1_1.csv	 'Part2 (1).ipynb'   test.csv


In [3]:
import numpy as np
import pandas as pd
from ast import literal_eval 
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import regexp_replace,col,array_contains,explode
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.functions import concat_ws
from pyspark.ml.classification import LogisticRegression

In [4]:
pd_dataFrame = pd.read_csv(r'train.csv')
data_spark_dataFrame = spark.createDataFrame(pd_dataFrame)

pd_dataFrame['genre']= pd_dataFrame['genre'].apply(literal_eval)
all_genre = pd_dataFrame['genre'].to_list()

In [5]:

labels =['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white','Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction','Mystery','Romantic comedy']
mat = np.zeros((len(pd_dataFrame),len(labels)))
for i,genre in enumerate(all_genre):
  for j,g in enumerate(genre):
    for k,label in enumerate(labels):
        if label==g:
          mat[i][k] = 1



In [7]:
labels = "Drama , Comedy , Romance Film , Thriller , Action , World cinema , Crime Fiction , Horror , Black-and-white , Indie , Action/Adventure , Adventure , Family Film , Short Film , Romantic drama , Animation , Musical , Science Fiction , Mystery , Romantic comedy"
np.savetxt("genre_lables.csv", mat, delimiter=",",fmt='%d',header=labels)


In [8]:
lables_dataFrame = pd.read_csv(r'genre_lables.csv')
test_pd_dataFrame = pd.read_csv(r'test.csv')


In [9]:
lables_spark_dataFrame = spark.createDataFrame(lables_dataFrame)
test_spark_dataFrame = spark.createDataFrame(test_pd_dataFrame)

In [10]:
ddataFrame1 = data_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
ddataFrame2 = lables_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
ddataFrame3 = test_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
dataFrame = ddataFrame1.join(ddataFrame2, "row_id").drop("row_id")
test_dataFrame = ddataFrame3.join(ddataFrame2, "row_id").drop("row_id")


In [13]:
regTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)
pipeline = Pipeline(stages=[regTokenizer, remover, hashingTF,idf])


In [14]:
Mdl = pipeline.fit(dataFrame)
dataset = Mdl.transform(dataFrame)

Mdl2 = pipeline.fit(test_dataFrame)
test_dataset = Mdl2.transform(test_dataFrame)

In [15]:
dfList = []
labelColumns = lables_spark_dataFrame.columns
logReg = LogisticRegression(featuresCol = 'features',maxIter=500)


In [16]:
for labelColumn in labelColumns:
    logReg.setLabelCol(labelColumn)
    logRegModel = logReg.fit(dataset)
    pred = logRegModel.transform(test_dataset)
    pred = pred.withColumn("prediction",F.col("prediction").cast(IntegerType()))
    dfList.append(pred.select("movie_id","prediction"))

In [17]:
dataFrames_renamed = [dataFrame.selectExpr('movie_id', f'prediction as prediction_{i}') for i, dataFrame in enumerate(dfList)]
temp_dataFrame = reduce(lambda x, y: x.join(y, ['movie_id'], how='full'), dataFrames_renamed)
col_list = ['prediction_%d' % i for i in range(len(dfList))]
temp_dataFrame = temp_dataFrame.withColumn('predictions',concat_ws(" ",*col_list)).drop(*col_list).toPandas().to_csv("predictions_part2.csv",index=False)